In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras.metrics as metrics
from keras.layers import Dense, LSTM, Bidirectional, GRU, Conv1D
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
cols = {
    
    # 'ts': int,
    # 'src_ip': 'category',
    'src_port': float,
    # 'dst_ip': 'category',
    'dst_port': float,
    'proto': 'category',
    # 'service': 'category',
    # 'duration': float,
    # 'src_bytes': float,
    # 'dst_bytes': float,
    # 'conn_state': 'category',
    # 'missed_bytes': float,

    # 'src_pkts': float,
    # 'src_ip_bytes': float,
    # 'dst_pkts': float,
    # 'dst_ip_bytes': float,

    # 'dns_query': 'category',
    # 'dns_qclass': float,
    # 'dns_qtype': float,
    # 'dns_rcode': float,
    # 'dns_AA': float, #bool
    # 'dns_RD': float, #bool
    # 'dns_rejected': float, #bool

    # 'ssl_version': 'category',
    # 'ssl_cipher': 'category',
    # 'ssl_resumed': float, #bool
    # 'ssl_established': float, #bool
    # 'ssl_subject': 'category',
    # 'ssl_issuer': 'category',

    # 'http_trans_depth': float,
    # 'http_method': 'category',
    # 'http_uri': 'category',
    # 'http_version': 'category',
    # 'http_request_body_len': float,
    # 'http_response_body_len': float,
    # 'http_status_code': float,
    # 'http_user_agent': 'category',
    # 'http_orig_mime_types': 'category',
    # 'http_resp_mime_types': 'category',

    # 'weird_name': 'category',
    # 'weird_addl': 'category',
    # 'weird_notice': float, #bool

    'label': int

}

def bool_to_number(x: str) -> float:
  if x == 'T':
    return 1
  elif x == 'F':
    return 0
  elif x == '-':
    return 0.5
  else:
    raise x

converters = {
    'dns_AA': bool_to_number,
    'dns_RD': bool_to_number,
    'dns_rejected': bool_to_number,
    'ssl_resumed': bool_to_number,
    'ssl_established': bool_to_number,
    'weird_notice': bool_to_number
}

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/datasci/ton_iot/Train_Test_Network.csv',
                      sep=',',
                      header=0,
                      dtype=cols,
                      usecols=list(cols.keys()),
                      converters=converters,
                      na_values=['-'])

In [ ]:
df

,src_port,dst_port,proto,label
0,1883.0,52976.0,tcp,0
1,47260.0,15600.0,udp,0
2,1880.0,51782.0,tcp,0
3,34296.0,10502.0,tcp,0
4,46608.0,53.0,udp,0
...,...,...,...,...
461038,33108.0,80.0,tcp,1
461039,37242.0,443.0,tcp,1
461040,4444.0,49178.0,tcp,1
461041,60816.0,443.0,tcp,1


In [ ]:
features = list(df.columns)
features.remove('label')

In [ ]:
for col,typ in dict(df.dtypes).items():
  if typ == 'category':
    df[col] = df[col].cat.codes.astype(int)
  else:
    vs = list(df[col].unique())
    # should not happen in the full dataset, otherwise would be removed in 1st step of preproc
    if (len(vs) == 1) and (np.isnan(vs[0])):
      print('error',col,'const column with NaN. Replacing with 0')
      df[col] = 0
    elif (len(vs) == 2) and (np.isnan(vs[0]) or np.isnan(vs[1])):
      val = vs[1] if np.isnan(vs[0]) else vs[0]
      val = -val if val != 0 else 1
      df[col] = df[col].fillna(value=val)
    else:
      df[col] = df[col].fillna(df[col].mean())

In [ ]:
X = df.drop('label',axis=1).values
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
wsize = 100
I = np.zeros((len(X), wsize, len(X[0])))
for i in range(len(X)):
  for j in range(wsize):
    if i >= j:
      I[i][j] = X[i-j]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(I, 
                                                    df['label'].values, 
                                                    test_size=0.3,
                                                    random_state=42,
                                                    shuffle=True)

In [ ]:
model = tf.keras.models.Sequential()

model.add(Bidirectional(GRU(64, activation='tanh', kernel_regularizer='l2')))
model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
model.add(Dense(1, activation = 'sigmoid', kernel_regularizer='l2'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy', metrics.Precision(), metrics.Recall(), metrics.AUC() ])

In [ ]:
history = model.fit(X_train,
                    Y_train,
                    epochs=20,
                    verbose = 1)

Epoch 1/20
10086/10086 [==============================] - 745s 73ms/step - loss: 0.1780 - accuracy: 0.9630 - precision: 0.9315 - recall: 0.9649 - auc: 0.9857
Epoch 2/20
10086/10086 [==============================] - 736s 73ms/step - loss: 0.1137 - accuracy: 0.9778 - precision: 0.9557 - recall: 0.9820 - auc: 0.9936
Epoch 3/20
10086/10086 [==============================] - 735s 73ms/step - loss: 0.0807 - accuracy: 0.9849 - precision: 0.9666 - recall: 0.9910 - auc: 0.9974
Epoch 4/20
10086/10086 [==============================] - 732s 73ms/step - loss: 0.0982 - accuracy: 0.9790 - precision: 0.9611 - recall: 0.9795 - auc: 0.9964
Epoch 5/20
10086/10086 [==============================] - 727s 72ms/step - loss: 0.1687 - accuracy: 0.9663 - precision: 0.9383 - recall: 0.9671 - auc: 0.9874
Epoch 6/20
10086/10086 [==============================] - 743s 74ms/step - loss: 0.1332 - accuracy: 0.9795 - precision: 0.9557 - recall: 0.9870 - auc: 0.9949
Epoch 7/20
10086/10086 [============================

In [ ]:
predict = model.predict(X_test, verbose=1)

4323/4323 [==============================] - 68s 16ms/step


In [ ]:
tp,tn,fp,fn = 0,0,0,0
predictn = predict.flatten().round().tolist()
len(predictn)
for i in range(len(predictn)):
  if predictn[i]==1 and Y_test[i]==1:
    tp+=1
  elif predictn[i]==0 and Y_test[i]==0:
    tn+=1
  elif predictn[i]==0 and Y_test[i]==1:
    fp+=1
  elif predictn[i]==1 and Y_test[i]==0:
    fn+=1
    
print(tp,tn,fp,fn)
classification_report(Y_test, predictn, output_dict=True)

48069 89253 272 719


{'0': {'f1-score': 0.9944790163623906,
  'precision': 0.9969617425300196,
  'recall': 0.9920086249055262,
  'support': 89972},
 '1': {'f1-score': 0.9897970739943784,
  'precision': 0.9852627695334918,
  'recall': 0.9943733063031381,
  'support': 48341},
 'accuracy': 0.9928350914230767,
 'macro avg': {'f1-score': 0.9921380451783846,
  'precision': 0.9911122560317557,
  'recall': 0.9931909656043322,
  'support': 138313},
 'weighted avg': {'f1-score': 0.9928426569745378,
  'precision': 0.9928729001679485,
  'recall': 0.9928350914230767,
  'support': 138313}}

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 128)              26496     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 43,137
Trainable params: 43,137
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.save('/content/drive/Shareddrives/datasci/ton_iot/GRU.tf')
with open('/content/drive/Shareddrives/datasci/ton_iot/result/GRU.csv','w') as infile:
  txt = ''
  for pred in predictn:
    txt += str(int(pred))+','
  txt = txt[:-1]
  txt += '\n'
  infile.write(txt)

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/datasci/ton_iot/GRU.tf/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/datasci/ton_iot/GRU.tf/assets
